In [1]:
import numpy as np
import threading
import queue
import pandas as pd
#testArray = np.array([[1,2,3,4],[2,3,4,1],[3,4,1,2],[4,1,2,3],[3,4,1,2],[4,1,2,3]])
testArray = np.array([[1, 2], [2, 1]])
np.shape(testArray)

(2, 2)

In [6]:
def simpleNMF(A, k, i):
    m, n = np.shape(A)
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)
    for _ in range(i):
        W = W * (A @ H.T) / (W @ H @ H.T)
        H = H * (W.T @ A) / (W.T @ W @ H)
    return W @ H

simpleNMF(testArray, 2, 1000)

array([[1., 2.],
       [2., 1.]])

In [7]:
def thread_function_W(A, W, H, q, i):
    q.put((i, W * (A @ H.T) / (W @ H @ H.T)))

def thread_function_H(A, W, H, q, i):
    q.put((i, H * (W.T @ A) / (W.T @ W @ H)))

def naiveParallelNMF(A, k, p, numIter):
    m, n = np.shape(A)
    if m % p > 0:
        raise TypeError('Input first dimension not divisible by number of threads')
    if n % p > 0:
        raise TypeError('Input second dimension not divisible by number of threads')
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)
    As1 = [A[int(i*m/p):int((i+1)*m/p), :] for i in range(p)]
    As2 = [A[:, int(i*n/p):int((i+1)*n/p)] for i in range(p)]
    
    for _ in range(numIter):
        Ws = [W[int(i*m/p):int((i+1)*m/p), :] for i in range(p)] # chop up W
        w_threads = []
        w_queue = queue.Queue()
        for j in range(p):
            newThread = threading.Thread(target = thread_function_W, args = (As1[j], Ws[j], H, w_queue, j)) # each thread updates one section
            newThread.start()
            w_threads.append(newThread)
        for thread in w_threads: # wait for all threads to complete
            thread.join()
        while not w_queue.empty(): # reconstitute and update W
            i, v = w_queue.get()
            Ws[i] = v
        W = np.concatenate(Ws, 0)

        Hs = [H[:, int(i*n/p):int((i+1)*n/p)] for i in range(p)] # same procedure for H
        h_threads = []
        h_queue = queue.Queue()
        for j in range(p):
            newThread = threading.Thread(target = thread_function_H, args = (As2[j], W, Hs[j], h_queue, j))
            newThread.start()
            h_threads.append(newThread)
        for thread in h_threads:
            thread.join()
        while not h_queue.empty():
            i, v = h_queue.get()
            Hs[i] = v
        H = np.concatenate(Hs, 1)
    return W @ H
    
naiveParallelNMF(testArray, 2, 2, 10000)

array([[1., 2.],
       [2., 1.]])

In [8]:
def HPC_NMF(A, k, p1, p2, numIter):
    m, n = np.shape(A)
    if m % (p1*p2) > 0:
        raise TypeError('Input first dimension not divisible by number of threads')
    if n % (p1*p2) > 0:
        raise TypeError('Input second dimension not divisible by number of threads')
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)

    Hs = [[np.random.rand(k, int(n/(p1*p2))) for i in range(p1)] for j in range(p2)]


    return A

Next, we will optimize both the number of processor dimensions (p) and the rank of the factorization (k). For each, we will optimize runtime, minimize computing costs, bandwith cost(?), and communication cost (?). We will examine these results on both dense and sparse imput matricies (A)(?). For each set, the study fixes the number of processors at 600 and vary the rank of k. We will replicate this, but also try to optimize the number of processors for our dataset.

Note** we need to find datasets; preferably one sparse and one dense dataset, or we could generate synthetic sparse and dense datasets.



In [15]:
c4 = pd.read_csv('c4_game_database.csv')
c4[c4 < 0] = 2 #changing -1 values to 2
c4
#really not sure if this works for a sparse dataset but wanted to start exploring

,pos_01,pos_02,pos_03,pos_04,pos_05,pos_06,pos_07,pos_08,pos_09,pos_10,...,pos_34,pos_35,pos_36,pos_37,pos_38,pos_39,pos_40,pos_41,pos_42,winner
0,1.0,1.0,1.0,2.0,2.0,1.0,0.0,2.0,2.0,2.0,...,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0
1,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,1.0,2.0
4,0.0,2.0,2.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,2.0,2.0,1.0,2.0,1.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376635,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,1.0,2.0,2.0,2.0,1.0,2.0,0.0,1.0,2.0
376636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0,2.0
376637,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0
376638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,2.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,0.0,2.0
